## Tarea 2

INTEGRANTES: JOSÉ MIGUEL MORALES JURE - 
             JOSÉ MANUEL JEREZ DÍAZ

- Usando las funciones anteriores, o desarrollando las propias, escriba una función que pueda valorizar, por simulación de Montecarlo, calls y puts sobre bonos cero cupón con el modelo de HW. Considere puts y calls de plazo máximo igual a 1Y.
- Su función debe poder recibir la semilla.
- Compare sus resultados con las fórmulas del modelo y muestre que diferencias obtiene usando 1000 simulaciones y una semilla predeterminada.
- Valorice también por simulación de Montecarlo el bono cupón cero a 1Y. Muestre que diferencias obtiene respecto al valor de mercado del bono usando 1000 simulaciones y una semilla predeterminada.

In [1]:
from finrisk import QC_Financial_3 as Qcf
from typing import List, Tuple, Callable
from scipy.interpolate import interp1d

import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

# from modules
import auxiliary as aux
from numpy import random as rnd
import plotly.express as px
import pandas as pd
import numpy as np
import math
from numpy import random as rnd
from scipy.stats import norm

In [2]:
datos =pd.read_excel("../data/20201012_built_sofr_zero.xlsx")
datos=pd.DataFrame(datos)
datos['t'] = datos['plazo'] / 365.0
datos['rate'] = np.log(1 / datos['df'])/( datos['plazo'] / 365.0)
zcurva = interp1d(datos['t'],
                  datos['rate'],
                  kind='cubic',
                  fill_value="extrapolate")
datos

,plazo,tasa,df,t,rate
0,1,0.000811,0.999998,0.002740,0.000811
1,7,0.000841,0.999984,0.019178,0.000841
2,14,0.000780,0.999970,0.038356,0.000780
3,21,0.000774,0.999955,0.057534,0.000774
4,33,0.000781,0.999929,0.090411,0.000781
5,61,0.000781,0.999870,0.167123,0.000781
6,92,0.000811,0.999796,0.252055,0.000811
7,125,0.000781,0.999733,0.342466,0.000781
8,152,0.000760,0.999683,0.416438,0.000760
9,182,0.000750,0.999626,0.498630,0.000750


In [3]:
def zrate(t: float) -> float:
    return zcurva(t)


def dzrate(t: float) -> float:
    delta = .0001
    return (zcurva(t + delta) - zrate(t)) / delta


def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta

In [4]:
def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

MUESTRA RESULTADOS DE ZRATE - SUS DERIVADAS y theta

In [5]:
t_opcion =1
T_bono = 2
t=0
sigma = 0.015
gamma = 0.5
# Proxy de la tasa instantánea r(t) es una tasa entre t y t + dt (donde dt es un intervalo infitesimal)
r0 = zrate(0)
K=1
dt=1/365

print(f'zrate({t:.2f}) = {zrate(t):.4%}')
print(f'dzrate({t:.2f}) = {dzrate(t):.4%}')
print(f'd2zrate({t:.2f}) = {d2zrate(t):.4%}')

def theta(t: float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux

zrate(0.00) = 0.0788%
dzrate(0.00) = 0.9560%
d2zrate(0.00) = -91.8498%


FUNCIONES PARA CALCULO DEL CUPON

In [6]:
def b_hw(gamma: float, t_opcion: float, T_bono: float) -> float:
    """
    Calcula el valor de la función B(t,T) que interviene en la fórmula
    para el valor de un bono cupón cero en el modelo de HW.
    params:
    - gamma: intensidad de reversión del modelo HW
    - t:
    - T:
    return:

    - valor de la función B(t, T)
    """
    aux = 1 - math.exp(- gamma * (T_bono - t_opcion))
    return aux / gamma

In [7]:
def a_hw(zrate: float, fwd, gamma: float, sigma: float, t_opcion: float, T_bono: float,
         verbose = False):
    """
    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t_opcion, T_bono)
    dfT = math.exp(-zrate(T_bono) * T_bono)
    dft = math.exp(-zrate(t_opcion) * t_opcion)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t_opcion)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t_opcion))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

In [8]:
def zero_hw(r: float, gamma: float, sigma: float,
            zrate: float, fwd: float, t_opcion: float, T_bono: float) -> float:
    a = a_hw(zrate, fwd, gamma, sigma, t_opcion, T_bono)
    b = b_hw(gamma, t_opcion, T_bono)
    return math.exp(a - b * r)
zero_hw(r0,gamma,sigma,zrate,fwd,t_opcion,T_bono)

0.9993919760351754

## CALL Y PUT FÓRMULA CERRADA

In [9]:
#Se definen las funciones de las que dependen Call y Put
# Strike
def s_z(gamma,t_opcion,T_bono,sigma):
    b = b_hw(gamma, t_opcion, T_bono)
    sz=b*np.sqrt((sigma**2/(2*gamma))*(1-np.exp(-2*gamma*t_opcion)))
    return sz
s_z(gamma,t_opcion,T_bono,sigma)

0.009384953162988194

In [10]:
def d1(t_opcion,T_bono,sigma,gamma,r,K):
    sz=s_z(gamma,t_opcion,T_bono,sigma)
    z=zero_hw(r,gamma,sigma,zrate,fwd,0,T_bono)
    z1=zero_hw(r,gamma,sigma,zrate,fwd,0,t_opcion)
    d_1=(np.log(z/(K*z1))+ 0.5*(sz**2))/sz
    return d_1
d1(t_opcion,T_bono,sigma,gamma,r0,K)

-0.04142166019983301

In [11]:
def d2(t_opcion,T_bono,sigma,gamma,r,K):
    d=d1(t_opcion,T_bono,sigma,gamma,r,K)
    sz=s_z(gamma,t_opcion,T_bono,sigma)
    d2=d-sz
    return d2
d2(t_opcion,T_bono,sigma,gamma,r0,K)

-0.050806613362821204

In [12]:
def Call_Put (K,r,gamma,sigma,zrate,fwd,t_opcion,T_bono):
    
    z=zero_hw(r,gamma,sigma,zrate,fwd,t,t_opcion)
    z1=zero_hw(r,gamma,sigma,zrate,fwd,t,T_bono)
    
    d=d1(t_opcion,T_bono,sigma,gamma,r,K)
    d_2=d2(t_opcion,T_bono,sigma,gamma,r,K) 
    
    Call=z*(((z1/z)*norm.cdf(d)) - K*norm.cdf(d_2))
    Put=z*((K* norm.cdf(-d_2))-((z1/z)*norm.cdf(-d)))
    return Call,Put
Call_Put(K,r0,gamma,sigma,zrate,fwd,t_opcion,T_bono)



(0.0035283883401179164, 0.003960769916741911)

## SIMULACIÓN DE R Y CERO CUPON


In [13]:
def theta_fwd_measure(t_opcion: float, T_bono:float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t_opcion))
    return dfwd(t_opcion) + gamma * fwd(t_opcion) + aux - b_hw(gamma, t_opcion, T_bono)*sigma**2

In [14]:
# Calculan las r para cada paso y se calcula el promedio
num_sim = 1000
num_steps = 264
seed = 1234
dt = 1 / 264.0
def sim_hw_many(gamma, sigma, theta_fwd_measure,t_opcion, r0, num_sim, num_steps, seed):
    """
    """
        
    num_steps += 1
    
    # Calcula los números aleatorios
    alea = np.zeros((num_sim, num_steps))
    np.random.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = np.random.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    tiempo = np.zeros(num_steps)
    for i in range(1, num_steps):
        tiempo[i] = i * dt
        theta_array[i] = theta_fwd_measure(i * dt,t_opcion)
    
    # Simula las trayectorias
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
    return sim
simul=sim_hw_many(gamma, sigma, theta_fwd_measure,t_opcion,r0,num_sim,num_steps,seed)
r=np.mean(simul)
print(f"Promedio de simulaciones de r: {r: .8}")

Promedio de simulaciones de r:  0.00033599685


Se comprueba que el z de la curva y z simulado son cercanos
### SIMULACIÓN DE FACTOR DE DESCUENTO EZ 

In [15]:
# valor del factor de descuento simulado 
def disc_factor(simul,dt):
    df=0
    for sim in simul:
        df  += math.exp(-dt * np.sum(sim))
    ez = df / num_sim
    z_curva = datos['df'].iloc[15]
    return ez,z_curva,abs(z_curva-ez)
a=disc_factor(simul,dt)
print(f"ez: {a[0]: .8%}")
print(f"z_curva: {a[1]: .8%}")
print(f"Diferencia discount factor: {a[2]: .8}")

ez:  99.96894692%
z_curva:  99.92978688%
Diferencia discount factor:  0.00039160039


## SIMULACIÓN CALL-PUT

In [16]:
#CON MAS SIMULACIONES, LOS VALORES SE ACERCAN CADA VEZ MAS.
num_sim = 1000
num_steps = 263
seed = 1234
dt=1/264

#Simulación Call y Put- Payoff
def Simulación(gamma,sigma,zrate,fwd,theta_fwd_measure,t_opcion,T_bono,r0,num_sim,num_steps,seed):
    df=0
    simul=sim_hw_many(gamma, sigma, theta_fwd_measure,t_opcion,r0,num_sim,num_steps,seed)
    r=simul[:,-1]
    z=[]
    Call_simulada=[]
    Put_simulada=[]
    Call_simul=[]
    Put_simul=[]
    for i in range (len(r)):
        zero=zero_hw(r[i],gamma,sigma,zrate,fwd,t_opcion,T_bono)
        z.append(zero)
        #Payoff
        C=max(z[i]-K,0)
        Call_simulada.append(C)
        P=max(K-z[i],0)
        Put_simulada.append(P)
    
    # valor del factor de descuento y Valor presente
    for sim in r:
        df += math.exp(-dt * np.sum(sim))
        #Payoff_C=Call_simulada[sim]*df
        #Call_simul.append(Payoff_C)
        #Payoff_P=Put_simulada[sim]*df
        #Put_simul.append(Payoff_P)
    df_final=df/len(r)
    print(f"ez: {df_final: .8%}")
    #print(f"Call: {np.mean(Call_simul): .8%}")
    #print(f"Put: {np.mean(Put_simul): .8%}")
    # VALOR PRESENTE
    for c in range(len(Call_simulada)):
        Call_1=Call_simulada[c]*df_final
        Call_simul.append(Call_1)
        Put_1=Put_simulada[c]*df_final
        Put_simul.append(Put_1)
    return df_final,np.mean(Call_simul),np.mean(Put_simul)

Simulation=Simulación(gamma,sigma,zrate,fwd,theta_fwd_measure,t_opcion,T_bono,r0,num_sim,num_steps,seed)
Simulation

ez:  99.99990388%


(0.9999990388105036, 0.0034872449710222565, 0.003635439288750601)

### COMPARACIÓN CALL-PUT SIMULADA VS FÓRMULA CERRADA


In [17]:
# Se demuestra el valor de la call y put con simulación y fórmula cerrada 
#Se calcula el error de simulación en base a la fórmula cerrada

# Formula cerrada
price_cerrada=Call_Put(K,r0,gamma,sigma,zrate,fwd,t_opcion,T_bono)
#Simulación
price_simulacion=Simulation
print('El valor de la Call con formula cerrada es:',price_cerrada[0])
print('El valor de la Call con simulación es:',Simulation[1])
print('El valor del error de simulación versus fórmula cerrada Call:',price_cerrada[0]-Simulation[1])
print('El valor de la Put con formula cerrada es:',price_cerrada[1])
print('El valor de la Put con simulación es:',Simulation[2])
print('El valor del error de simulación versus fórmula cerrada Put:',price_cerrada[1]-Simulation[2])

El valor de la Call con formula cerrada es: 0.0035283883401179164
El valor de la Call con simulación es: 0.0034872449710222565
El valor del error de simulación versus fórmula cerrada Call: 4.114336909565985e-05
El valor de la Put con formula cerrada es: 0.003960769916741911
El valor de la Put con simulación es: 0.003635439288750601
El valor del error de simulación versus fórmula cerrada Put: 0.00032533062799130997


### Tests

In [18]:
import hull_white as hw

In [22]:
hw.zcb_call_put(
    hw.CallPut.CALL,
    strike=K,
    to=t_opcion,
    tb=T_bono,
    r0=zcurva(0),
    zo=math.exp(-zcurva(t_opcion) * t_opcion),
    zb=math.exp(-zrate(T_bono) * T_bono),
    gamma=gamma,
    sigma=sigma)

0.0035283883401179272

In [23]:
hw.zcb_call_put(
    hw.CallPut.PUT,
    strike=K,
    to=t_opcion,
    tb=T_bono,
    r0=zcurva(0),
    zo=math.exp(-zcurva(t_opcion) * t_opcion),
    zb=math.exp(-zrate(T_bono) * T_bono),
    gamma=gamma,
    sigma=sigma)

0.00396076991674188

**AD:** verificar más casos ... lo mismo para el factor de descuento.